In [14]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [15]:
import pandas as pd
import mappings as mappings
import schemas as schemas
import datetime
import time
import glob
import os

# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/pre-processing/*")
for f in files:
    os.remove(f)

start_time = time.time()
current_year = 2022
accounts_return_period_start_date = datetime.date(current_year - 1, 9, 10)
academy_year_start_date = datetime.date(current_year - 1, 9, 1)
academy_year_end_date = datetime.date(current_year, 8, 30)
maintained_schools_year_start_date = datetime.date(current_year, 4, 1)
maintained_schools_year_end_date = datetime.date(current_year, 3, 31)

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [16]:
cdc = pd.read_csv('data/standing_data_cdc.csv', encoding='utf8', index_col=schemas.cdc_index_col, usecols=schemas.cdc.keys(), dtype=schemas.cdc)

cdc['Total Internal Floor Area'] = cdc.groupby(by=['URN'])['GIFA'].sum()
cdc['Proportion Area'] = (cdc['GIFA'] / cdc['Total Internal Floor Area'])
cdc['Indicative Age'] = cdc['Block Age'].fillna('').map(mappings.map_block_age).astype('Int64')
cdc['Age Score'] = cdc['Proportion Area'] * (current_year - cdc['Indicative Age'])
cdc['Age Average Score'] = cdc.groupby(by=['URN'])['Age Score'].sum()
cdc = cdc[['Total Internal Floor Area', 'Age Average Score']].drop_duplicates()

In [17]:
cdc.to_csv('output/pre-processing/cdc.csv')
cdc

,Total Internal Floor Area,Age Average Score
URN,,
100150,2803.0,48.358188
100162,2105.0,133.162945
100164,2934.0,97.0
100166,2040.0,91.705882
105304,1602.0,35.752809
...,...,...
144913,3111.0,16.704275
144917,2620.0,78.412214
105623,3382.0,7.0


## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [18]:
school_workforce_census = pd.read_excel('data/School_Tables_School_Workforce_Census_2022.xlsx', header=5, index_col=schemas.workforce_census_index_col, usecols=schemas.workforce_census.keys(), dtype=schemas.workforce_census, na_values=["x","u","c"], keep_default_na=True).drop_duplicates()

school_pupil_census = pd.read_csv('data/standing_data_census_pupils.csv', encoding='utf8', index_col=schemas.pupil_census_index_col, usecols=schemas.pupil_census.keys(), dtype=schemas.pupil_census).drop_duplicates()

census = school_pupil_census.join(school_workforce_census, on='URN', how='inner', rsuffix='_pupil', lsuffix='_workforce')

census.drop(labels=['full time pupils', 'headcount of pupils'], axis=1, inplace=True)
         

In [19]:

# Rename Columns
census.rename(columns={
    "Total Number of Non-Classroom-based School Support Staff, (Other school support staff plus Administrative staff plus Technicians and excluding Auxiliary staff (Full-Time Equivalent)": "FullTimeOther",
    "Total Number of Non Classroom-based School Support Staff, Excluding Auxiliary Staff (Headcount)": "FullTimeOtherHeadCount",
}, inplace=True)

In [20]:
census.to_csv('output/pre-processing/census.csv')
census

,% of pupils known to be eligible for and claiming free school me,% of pupils known to be eligible for free school meals (Performa,number of pupils whose first language is known or believed to be other than English,Statutory Low Age,Total School Workforce (Headcount),Total Number of Teachers in the Leadership Group (Headcount),Total Number of Teachers (Headcount),Total Number of Teaching Assistants (Headcount),FullTimeOtherHeadCount,Total Number of Auxiliary Staff (Headcount),Total School Workforce (Full-Time Equivalent),Total Number of Teachers in the Leadership Group (Full-time Equivalent),Total Number of Teachers (Full-Time Equivalent),Total Number of Teaching Assistants (Full-Time Equivalent),FullTimeOther,Total Number of Auxiliary Staff (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Teachers with Qualified Teacher Status (%) (Headcount),Number of Vacant Teacher Posts
URN,,,,,,,,,,,,,,,,,,,
141334,33.8,52.3,93.0,4,48,3,15,14,6,13,34.17,2.64,13.11,10.29,4.82,5.95,24.8,100.000000,0
141396,23.4,60.3,236.0,3,118,4,39,34,11,34,82.47,4.00,34.00,29.55,10.13,8.79,18.3,100.000000,0
141397,33.2,47.7,127.0,3,105,5,27,42,9,27,72.81,4.24,24.55,31.84,6.55,9.87,19.7,100.000000,0
142223,5.1,8.7,343.0,3,156,5,56,44,9,47,99.66,4.16,47.12,33.07,6.57,12.90,23.0,100.000000,0
144396,56.7,64.8,29.0,3,37,2,13,9,4,11,25.57,2.00,11.39,7.36,4.00,2.82,18.1,100.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104642,2.4,2.6,14.0,4,52,4,18,10,6,18,34.47,3.60,15.80,6.27,5.22,7.18,26.6,100.000000,0
104643,3.5,8.5,13.0,3,68,3,19,24,6,19,39.89,3.00,17.40,11.74,3.34,7.41,24.7,100.000000,0
104645,32.9,33.8,43.0,7,37,3,13,10,4,10,26.47,3.00,12.40,6.78,3.19,4.10,19.1,92.307692,0


## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [21]:
sen = pd.read_csv('data/SEN.csv', encoding='cp1252', index_col=schemas.sen_index_col, dtype=schemas.sen, usecols=schemas.sen.keys())
sen['Percentage SEN'] = (sen['EHC plan'] / sen['Total pupils']) * 100.0
sen['Primary Need SPLD'] = sen['EHC_Primary_need_spld'] + sen['SUP_Primary_need_spld']
sen['Primary Need MLD'] = sen['EHC_Primary_need_mld'] + sen['SUP_Primary_need_mld']
sen['Primary Need SLD'] = sen['EHC_Primary_need_sld'] + sen['SUP_Primary_need_sld']
sen['Primary Need PMLD'] = sen['EHC_Primary_need_pmld'] + sen['SUP_Primary_need_pmld']
sen['Primary Need SEMH'] = sen['EHC_Primary_need_semh'] + sen['SUP_Primary_need_semh']
sen['Primary Need SLCN'] = sen['EHC_Primary_need_slcn'] + sen['SUP_Primary_need_slcn']
sen['Primary Need HI'] = sen['EHC_Primary_need_hi'] + sen['SUP_Primary_need_hi']
sen['Primary Need VI'] = sen['EHC_Primary_need_vi'] + sen['SUP_Primary_need_vi']
sen['Primary Need MSI'] = sen['EHC_Primary_need_msi'] + sen['SUP_Primary_need_msi']
sen['Primary Need PD'] = sen['EHC_Primary_need_pd'] + sen['SUP_Primary_need_pd']
sen['Primary Need ASD'] = sen['EHC_Primary_need_asd'] + sen['SUP_Primary_need_asd']
sen['Primary Need OTH'] = sen['EHC_Primary_need_oth'] + sen['SUP_Primary_need_oth']
sen.rename(columns={'prov_slcn': 'Prov_SLCN', 'prov_hi':'Prov_HI', 'prov_vi':'Prov_VI', 'prov_msi': 'Prov_MSI', 'prov_pd':'Prov_PD', 'prov_asd':'Prov_ASD', 'prov_oth':'Prov_OTH'}, inplace=True)

sen = sen[['Total pupils', 'EHC plan', 'Percentage SEN', 'Primary Need SPLD', 'Primary Need MLD', 'Primary Need SLD', 'Primary Need PMLD', 'Primary Need SEMH', 'Primary Need SLCN', 'Primary Need HI', 'Primary Need VI', 'Primary Need MSI', 'Primary Need PD', 'Primary Need ASD', 'Primary Need OTH', 'Prov_SPLD', 'Prov_MLD', 'Prov_SLD', 'Prov_PMLD', 'Prov_SEMH', 'Prov_SLCN','Prov_HI','Prov_VI','Prov_MSI','Prov_PD','Prov_ASD','Prov_OTH']] 

In [22]:
sen.to_csv("output/pre-processing/sen.csv")
sen

,Total pupils,EHC plan,Percentage SEN,Primary Need SPLD,Primary Need MLD,Primary Need SLD,Primary Need PMLD,Primary Need SEMH,Primary Need SLCN,Primary Need HI,...,Prov_SLD,Prov_PMLD,Prov_SEMH,Prov_SLCN,Prov_HI,Prov_VI,Prov_MSI,Prov_PD,Prov_ASD,Prov_OTH
URN,,,,,,,,,,,,,,,,,,,,,
100000,271,8,2.95203,2,4,0,0,9,31,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100001,739,0,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,269,0,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,1045,0,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,136,2,1.470588,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149557,41,3,7.317073,2,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149632,1291,58,4.492641,31,15,0,0,20,25,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149633,86,0,0.0,2,1,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [23]:
ar_cell_mapping = pd.read_csv('data/AR_cell_mapping.csv', encoding='utf8', index_col=schemas.ar_cell_mapping_index_col, usecols=schemas.ar_cell_mapping.keys(), dtype=schemas.ar_cell_mapping)

ar_raw = pd.read_csv('data/AR_raw.csv', encoding='utf8', index_col=schemas.ar_index_col, usecols=schemas.ar.keys(), dtype=schemas.ar)

ar = ar_raw.reset_index().merge(ar_cell_mapping, right_on='cell', left_on='aruniquereference').set_index(schemas.ar_index_col)

pfi_schools = ar[ar['aruniquereference'] == 'BAE310-T'][['value']].map(mappings.map_is_pfi_school).rename(columns={'value': 'PFI School'})

academy_financial_position = ar[ar['aruniquereference'] == 'BAB030-T'][['value']].rename(columns={'value': 'Academy Balance'})
central_services_financial_position = ar[(ar['aruniquereference'] == 'BAB030-T') | (ar['aruniquereference'] == 'BTB030')].groupby('trustupin')['value'].sum().rename('Central Services Balance')
trust_financial_position = ar[(ar['aruniquereference'] == 'BTB030')][['trustupin', 'value']].rename(columns={'value': 'Trust Balance'}).set_index('trustupin')

teachingstaff = ar[ar['aruniquereference'] == 'BAE010-T'][['value']].rename(columns={'value': 'TeachingStaff'})
cateringsupplies = ar[ar['aruniquereference'] == 'BAE250-T'][['value']].rename(columns={'value': 'CateringSupplies'}) 


ar = (ar.join(pfi_schools, how='left')
      .join(academy_financial_position, how='left')
      .join(teachingstaff, how='left')
      .join(cateringsupplies, how='left')
      .join(trust_financial_position, on='trustupin', how='left')
      .join(central_services_financial_position, on='trustupin', how='left'))

In [24]:
ar.to_csv('output/pre-processing/ar.csv')
ar

,trustupin,companynumber,aruniquereference,value,Description L1,Metric,Metric ID,Cost Pool,Presentation name,Cost Pool ID,PFI School,Academy Balance,TeachingStaff,CateringSupplies,Trust Balance,Central Services Balance
academyupin,,,,,,,,,,,,,,,,
111443,137157,8146633,BAE010-T,5687.0,Staff costs,Teaching staff,1,Teaching and Teaching support staff,Teaching and Teaching support staff,1,Non-PFI school,59.0,5687.0,651.0,359.0,3506.0
111443,137157,8146633,BAE020-T,66.0,Staff costs,Supply teaching staff,5,Teaching and Teaching support staff,Teaching and Teaching support staff,1,Non-PFI school,59.0,5687.0,651.0,359.0,3506.0
111443,137157,8146633,BAE030-T,916.0,Staff costs,Education support staff,2,Teaching and Teaching support staff,Teaching and Teaching support staff,1,Non-PFI school,59.0,5687.0,651.0,359.0,3506.0
111443,137157,8146633,BAE040-T,623.0,Staff costs,Administrative and clerical staff,6,Non-educational support staff,Non-educational support staff,2,Non-PFI school,59.0,5687.0,651.0,359.0,3506.0
111443,137157,8146633,BAE050-T,390.0,Staff costs,Premises staff,14,Premises,Premises,5,Non-PFI school,59.0,5687.0,651.0,359.0,3506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
<NA>,136961,9040380,BTB030,136.0,Closing balance,Closing balance (Restricted and Unrestricted F...,0,<NA>,<NA>,0,NaN,NaN,NaN,NaN,136.0,2634.0
<NA>,141510,10269490,BTB030,0.0,Closing balance,Closing balance (Restricted and Unrestricted F...,0,<NA>,<NA>,0,NaN,NaN,NaN,NaN,0.0,6.0
<NA>,137157,8146633,BTB030,359.0,Closing balance,Closing balance (Restricted and Unrestricted F...,0,<NA>,<NA>,0,NaN,NaN,NaN,NaN,359.0,3506.0


Create a summary table for the AR stance of each distinct academy in the table.

In [25]:
academy_ar = ar.reset_index().drop_duplicates(subset=['academyupin'], ignore_index=True)[
    ['academyupin', 'Academy Balance', 'Trust Balance', 'Central Services Balance', 'PFI School', 'TeachingStaff', 'CateringSupplies']
].set_index('academyupin')

academy_ar['Central Services Financial Position'] = academy_ar['Central Services Balance'].map(mappings.map_is_surplus_deficit)
academy_ar['Academy Financial Position'] = academy_ar['Academy Balance'].map(mappings.map_is_surplus_deficit) 
academy_ar['Trust Financial Position'] = academy_ar['Trust Balance'].map(mappings.map_is_surplus_deficit) 

In [26]:
academy_ar

,Academy Balance,Trust Balance,Central Services Balance,PFI School,TeachingStaff,CateringSupplies,Central Services Financial Position,Academy Financial Position,Trust Financial Position
academyupin,,,,,,,,,
111443,59.0,359.0,3506.0,Non-PFI school,5687.0,651.0,Surplus,Surplus,Surplus
111451,0.0,1909.0,1909.0,Non-PFI school,3728.0,60.0,Surplus,Deficit,Surplus
111453,0.0,3873.0,3873.0,Non-PFI school,428.0,10.0,Surplus,Deficit,Surplus
111710,1442.0,NaN,1442.0,Non-PFI school,5609.0,179.0,Surplus,Surplus,Unknown
113087,1655.0,0.0,1655.0,Non-PFI school,5227.0,71.0,Surplus,Surplus,Deficit
...,...,...,...,...,...,...,...,...,...
163968,488.0,42.0,2672.0,Non-PFI school,355.0,41.0,Surplus,Surplus,Surplus
163969,0.0,0.0,0.0,Non-PFI school,1081.0,99.0,Deficit,Deficit,Deficit
163970,-115.0,-1127.0,1544.0,Non-PFI school,1194.0,43.0,Surplus,Deficit,Deficit


## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [30]:
gias = pd.read_csv('data/edubasealldata20240312.csv', encoding='cp1252', 
                   index_col=schemas.gias_index_col, usecols=schemas.gias.keys(), dtype=schemas.gias)

gias_links = pd.read_csv('data/links_edubasealldata20240312.csv', encoding='cp1252', 
                         index_col=schemas.gias_links_index_col, usecols=schemas.gias_links.keys(), dtype=schemas.gias_links)

# GIAS transformations
gias['LA Establishment Number'] = gias['LA (code)'] + '-' + gias['EstablishmentNumber'].astype('string')
gias['LA Establishment Number'] = gias['LA Establishment Number'].astype('string')

gias['OpenDate'] = pd.to_datetime(gias['OpenDate'], dayfirst=True, format='mixed')
gias['CloseDate'] = pd.to_datetime(gias['CloseDate'], dayfirst=True, format='mixed')
gias['SchoolWebsite'] = gias['SchoolWebsite'].fillna('').map(mappings.map_school_website)
gias['Boarders (name)'] = gias['Boarders (name)'].fillna('').map(mappings.map_boarders)
gias['OfstedRating (name)'] = gias['OfstedRating (name)'].fillna('').map(mappings.map_ofsted_rating)
gias['NurseryProvision (name)'] = gias['NurseryProvision (name)'].fillna('')
gias['OfficialSixthForm (name)'] = gias['OfficialSixthForm (name)'].fillna('').map(mappings.map_sixth_form)
gias['AdmissionsPolicy (name)'] = gias['AdmissionsPolicy (name)'].fillna('').map(mappings.map_admission_policy)
gias['HeadName'] = gias['HeadTitle (name)'] + ' ' + gias['HeadFirstName'] + ' ' + gias['HeadLastName']

In the following cell, we find all the predecessor and merged links. The links are then Ranked by URN and order by 'Link Established Date'. The linked GAIS data in then joined to the base GIAS data. This creates the overall school data set. This dataset is then filtered for schools that are open (CloseDate is null) and the schools with nested links that are Ranked 1.

In [31]:
gias_links = gias_links[
    gias_links['LinkType'].isin(['Predecessor', 
                                 'Predecessor - amalgamated', 
                                 'Predecessor - Split School', 
                                 'Predecessor - merged', 
                                 'Merged - expansion of school capacity', 
                                 'Merged - change in age range'])
].sort_values(by='LinkEstablishedDate', ascending=False)

gias_links['Rank'] = gias_links.groupby('URN').cumcount() + 1
gias_links['Rank'] = gias_links['Rank'].astype('Int64')

schools = gias.join(gias_links, on='URN', how='left', rsuffix='_links', lsuffix='_school').sort_values(by='URN')

schools = schools[
    schools['CloseDate'].isna() & ((schools['Rank'] == 1) | (schools['Rank'].isna()))
].drop(columns=['LinkURN', 'LinkName', 'LinkType', 'LinkEstablishedDate', 'Rank'])

In [ ]:
schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [ ]:
academies_list = pd.read_csv('data/master_list_raw.csv', encoding='utf8', index_col=schemas.academy_master_list_index_col, dtype=schemas.academy_master_list, usecols=schemas.academy_master_list.keys()).rename(columns={'UKPRN': 'Academy UKPRN'})

academies_base = academies_list.merge(schools.reset_index(), left_index=True, right_on='LA Establishment Number').set_index('URN')

academies = (academies_base.merge(census, on='URN', how='left')
             .merge(sen, on='URN', how='left')
             .merge(cdc, on='URN', how='left')
             .merge(academy_ar, left_on='Academy UPIN', right_index=True, how='left'))

academies['Type of Provision - Phase'] = academies.apply(lambda df: mappings.map_academy_phase_type(df['TypeOfEstablishment (code)'], df['Type of Provision - Phase']), axis=1)

# Bizarre I shouldn't need this as this is coming from the original GIAS dataset, but I seem to have to do this twice. 
academies['NurseryProvision (name)'] = academies['NurseryProvision (name)'].fillna('')
academies['NurseryProvision (name)'] = academies.apply(lambda df: mappings.map_nursery(df['NurseryProvision (name)'], df['Type of Provision - Phase']), axis=1)

academies['Status'] = academies.apply(lambda df: mappings.map_academy_status(pd.to_datetime(df['Date left or closed if in period']), 
                                                                             pd.to_datetime(df['Valid to']), 
                                                                             pd.to_datetime(df['OpenDate']), 
                                                                             pd.to_datetime(df['CloseDate']), 
                                                                             pd.to_datetime(accounts_return_period_start_date), pd.to_datetime(academy_year_start_date), pd.to_datetime(academy_year_end_date)), axis=1)

academies['SchoolPhaseType'] = academies.apply(lambda df: mappings.map_school_phase_type(df['TypeOfEstablishment (code)'], df['Type of Provision - Phase']), axis=1)

academies.rename(columns={
    'UKPRN_x':'UKPRN',
    'Number of Pupils': 'Number of pupils',
    '% of pupils known to be eligible for free school meals (Performa': 'Percentage Free school meals'
}, inplace=True)

In [ ]:
academies.to_csv('output/pre-processing/academies.csv')
academies.sort_index()

Merge required census and cdc data to the maintained schools data set

In [ ]:
# Load raw list from CSV
maintained_schools_list = pd.read_csv('data/maintained_schools_raw.csv', encoding='utf8', index_col=schemas.maintained_schools_master_list_index_col, usecols=schemas.maintained_schools_master_list.keys(), dtype=schemas.maintained_schools_master_list)

In [ ]:
# Merge maintained_schools_base with schools (metadata from GIAS) & rename PFI column
maintained_schools = maintained_schools_list.merge(schools.reset_index(), left_index=True, right_on='URN')


In [ ]:
# Merge in census and cdc data
maintained_schools = (maintained_schools
                      .merge(sen, on='URN', how='left')
                      .merge(census, on='URN', how='left')
                      .merge(cdc, on='URN', how='left'))



In [ ]:
# Compute columns
maintained_schools['PFI'] = maintained_schools['PFI'].map(lambda x: 'PFI school' if x == 'Y' else 'Non-PFI school')
maintained_schools['Status'] = maintained_schools.apply(lambda df: mappings.map_maintained_school_status(df['OpenDate'], df['CloseDate'], df['Period covered by return (months)'], pd.to_datetime(maintained_schools_year_start_date), pd.to_datetime(maintained_schools_year_end_date)), axis=1)
maintained_schools['School Balance'] = maintained_schools['Total Income   I01 to I18'] - maintained_schools['Total Expenditure  E01 to E32']
maintained_schools['School Financial Position'] = maintained_schools['School Balance'].map(mappings.map_is_surplus_deficit)
maintained_schools['SchoolPhaseType'] = maintained_schools.apply(lambda df: mappings.map_school_phase_type(df['TypeOfEstablishment (code)'], df['Overall Phase']), axis=1)
maintained_schools['Partial Years Present'] = maintained_schools['Period covered by return (months)'].map(lambda x: x != 12)
maintained_schools['Did Not Submit'] = maintained_schools['Did Not Supply flag'].map(lambda x: x == 1)

In [ ]:
# Compute Columns - Will
maintained_schools.columns.values

In [ ]:
# Rename columns
maintained_schools.rename(columns={
    'PFI': 'PFI School',
    '% of pupils eligible for FSM': 'Percentage Free school meals',
    'No Pupils': 'Number of pupils',
    'I07  Other grants and payments': 'Other grants and payments'
}, inplace=True)

In [ ]:
maintained_schools.set_index('URN', inplace=True)

In [ ]:
maintained_schools

In [ ]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv')


## Federation Capture




In [ ]:
group_links = pd.read_csv('data/alllinksdata20240417.csv', encoding='unicode-escape',  index_col=schemas.groups_index_col, usecols=schemas.groups.keys(), dtype=schemas.groups)

In [ ]:
# filter lead schools out of the maintained schools list
federations = maintained_schools[['LAEstab']][maintained_schools['Federation']=='Lead school'].copy()
# join 
federations = federations.join(group_links[['Group Name','Group UID','Closed Date']])


# remove federations with an associated closed date
federations = federations.loc[federations['Closed Date'].isna()]

# federations with a UID listed in the GIAS groups data are referred to as "Hard" federations
# while federations not listed in GIAS are referred to as "Soft" federations.
# Soft federation UIDs are a combination of their URN and LAEstab codes.

# create mask for soft federations
mask = federations['Group UID'].isna()

hard_federations = federations.loc[~mask].copy()
soft_federations = federations.loc[mask].copy()

# define members list for hard federations
group_links['Members'] = group_links.index
hard_members = group_links[['Members','Group UID']].groupby('Group UID').agg(list)

hard_federations = hard_federations.join(hard_members, on='Group UID')

# Rename columns
hard_federations.rename(columns={
    'Group Name': 'FederationName',
    'Group UID': 'FederationUid',
}, inplace=True)

# for the soft federations
soft_federations['FederationUid'] = soft_federations.index.astype(str) + soft_federations['LAEstab'].astype(str)

# Rename columns
soft_federations.rename(columns={
    'Group Name': 'FederationName',
    'Group UID': 'FederationUid',
}, inplace=True)

# TODO - add in soft federation members and names (currently no mapping available)

In [ ]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

In [ ]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations

## KS2 and KS4 performance

In [45]:
ks2 = pd.read_excel('data/2022-2023_england_ks2revised.xlsx',usecols=schemas.ks2.keys())




In [42]:
# ks2 mappings
ks2['READPROG'] = ks2['READPROG'].replace({
                                        'SUPP':0,
                                        'LOWCOV':0})
ks2['MATPROG']=ks2['MATPROG'].replace({
                                        'SUPP':0,
                                        'LOWCOV':0})
ks2['WRITPROG']=ks2['WRITPROG'].replace({
                                        'SUPP':0,
                                        'LOWCOV':0})

ks2['Ks2Progress'] = ks2['READPROG'].astype(float) + ks2['MATPROG'].astype(float)+ks2['WRITPROG'].astype(float)


#ks2 = ks2[['URN','LAEstab','Ks2Progress']].copy()

KeyError: 'LAE'

In [44]:
ks2[ks2['LEA']=='u']

,LEA,ESTAB,URN,READPROG,WRITPROG,MATPROG,Ks2Progress


In [38]:
ks2.to_csv('output/ks2.csv')
ks2

,URN,LAEstab,Ks2Progress
0,100000.0,201.03614.0,0.5
1,136807.0,202.02000.0,13.0
2,139837.0,202.02001.0,20.7
3,140686.0,202.02002.0,-1.8
4,100008.0,202.02019.0,5.5
...,...,...,...
16534,NaN,941.0nan,-0.1
16535,NaN,942.0nan,-2.0
16536,NaN,943.0nan,-0.7
16537,NaN,nannan,NaN


In [32]:
ks4 = pd.read_excel('data/2022-2023_england_ks4revised.xlsx')

In [33]:
#ks4 mappings

ks4['LAEstab'] = ks4['LEA'].astype(str) + ks4['ESTAB'].astype(str)

ks4.rename(columns={
    'ATT8SCR':'AverageAttainment',
    'P8MEA':'Progress8Measure',
    'P8_BANDING':'Progress8Banding'
})

ks4 = ks4[['URN','LAEstab','AverageAttainment','Progress8Measure','Progress8Banding']].copy()

,RECTYPE,LEA,ESTAB,URN,SCHNAME,SCHNAME_AC,ADDRESS1,ADDRESS2,ADDRESS3,TOWN,...,TAVENT_GFSM6CLA1A_PTQ_EE,TAVENT_GNFSM6CLA1A_PTQ_EE,TAVENT_GFSM6CLA1A_21_PTQ_EE,TAVENT_GNFSM6CLA1A_21_PTQ_EE,TAVENT_GFSM6CLA1A_22_PTQ_EE,TAVENT_GNFSM6CLA1A_22_PTQ_EE,TTOTENT_E_TOTAL_PTQ_EE,TTOTENT_E_COVID_IMPACTED_PTQ_EE,PTOTENT_E_COVID_IMPACTED_PTQ_EE,Progress8Banding
0,1,201.0,6007.0,100003.0,City of London School,NaN,107 Queen Victoria Street,NaN,NaN,London,...,NP,NP,NaN,NaN,NP,NP,569.0,3.0,1%,NaN
1,1,201.0,6005.0,100001.0,City of London School for Girls,NaN,St Giles' Terrace,Barbican,NaN,London,...,NP,NP,NaN,NaN,NP,NP,606.0,5.0,1%,NaN
2,1,201.0,6000.0,100544.0,David Game College,NaN,31 Jewry Street,London,NaN,NaN,...,NP,NP,NaN,NaN,NP,NP,53.0,1.0,2%,NaN
3,4,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,202.0,4285.0,100053.0,Acland Burghley School,NaN,Burghley Road,NaN,NaN,London,...,7.0,8.1,NaN,NaN,7,8.2,1397.0,0.0,0%,Average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5808,1,943.0,6008.0,112446.0,Windermere School,NaN,Browhead,Patterdale Road,Windermere,The Lake District,...,NP,NP,NaN,NaN,NP,NP,88.0,0.0,0%,NaN
5809,2,943.0,6053.0,134191.0,Wings School,NaN,Whassett,Milnthorpe,Cumbria,Milnthorpe,...,NP,NP,NaN,NaN,NaN,NaN,20.0,0.0,0%,NaN
5810,4,943.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,7.8,NaN,NaN,NaN,NaN,20290.0,2.0,0%,Below average
5811,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ks4.to_csv('output/ks4.csv')
ks4

### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')